In [6]:
from utils import getAllImageInfo

dir = './face'
all = getAllImageInfo(dir)
print('total image count', len(all[0]))

total image count 6040


In [7]:
from PIL import Image

aspectRatioTotal = 0

for imgName, fullPath in zip(all[0], all[1]):
    im = Image.open(fullPath)
    aspectRatioTotal += im.size[0] / im.size[1]

averageAspectRatio = aspectRatioTotal / len(all[0])
print('average acpect ratio is', averageAspectRatio)

average acpect ratio is 0.5925324227373069
